## Change `NUMPROC` variable to the actual number of cores

In [1]:
NUMPROC=30

## Set up location of data and reference files

In [17]:
base_dir="../data"
result_dir="../results"
index="../ref/lncRNA_locus"

## Perform quality trimming

* discard reads where average quality in 4 nt window falls below 20
* clip Illumina adapters
* keep only reads longer than 36 nt

Files `R1_pe.fastq.gz` and `R2_pe.fastq.gz` contain surviving paired end reads.
Files `R1_se.fastq.gz` and `R2_se.fastq.gz` containg reads where only one mate survived.

In [4]:
for sample in $(ls $base_dir)
do
    echo "Running Trimmomatic for $sample"
    dst="$base_dir/$sample/trim"
    if [ ! -d $dst ];
    then
        mkdir -p $dst
    fi
    TrimmomaticPE -threads $NUMPROC \
        $base_dir/$sample/R1.fastq.gz \
        $base_dir/$sample/R2.fastq.gz \
        $dst/R1_pe.fastq.gz $dst/R1_se.fastq.gz \
        $dst/R2_pe.fastq.gz $dst/R2_se.fastq.gz \
        ILLUMINACLIP:../ref/illuminaClipping.fa:2:30:10 \
        SLIDINGWINDOW:4:20 MINLEN:36
done

echo "DONE."

Running Trimmomatic for yoon01
TrimmomaticPE: Started with arguments:
 -threads 30 ../data/yoon01/R1.fastq.gz ../data/yoon01/R2.fastq.gz ../data/yoon01/trim/R1_pe.fastq.gz ../data/yoon01/trim/R1_se.fastq.gz ../data/yoon01/trim/R2_pe.fastq.gz ../data/yoon01/trim/R2_se.fastq.gz ILLUMINACLIP:../ref/illuminaClipping.fa:2:30:10 SLIDINGWINDOW:4:20 MINLEN:36
Using PrefixPair: 'GATCGGAAGAGCGGTTCAGCAGGAATGCCGAG' and 'ACACTCTTTCCCTACACGACGCTCTTCCGATCT'
ILLUMINACLIP: Using 1 prefix pairs, 0 forward/reverse sequences, 0 forward only sequences, 0 reverse only sequences
Quality encoding detected as phred33
Input Read Pairs: 35729030 Both Surviving: 30534716 (85.46%) Forward Only Surviving: 2021392 (5.66%) Reverse Only Surviving: 527614 (1.48%) Dropped: 2645308 (7.40%)
TrimmomaticPE: Completed successfully
Running Trimmomatic for yoon02
TrimmomaticPE: Started with arguments:
 -threads 30 ../data/yoon02/R1.fastq.gz ../data/yoon02/R2.fastq.gz ../data/yoon02/trim/R1_pe.fastq.gz ../data/yoon02/trim/R1_se

In [16]:
ls -lah ../data/yoon01/trim

total 1.3G
drwxrwsr-x 2 ilya lab 4.0K Feb  2 16:51 .
drwxrwsr-x 3 ilya lab 4.0K Feb  2 15:56 ..
-rw-rw-r-- 1 ilya lab 582M Feb  2 16:18 R1_pe.fastq.gz
-rw-rw-r-- 1 ilya lab  53M Feb  2 16:18 R1_se.fastq.gz
-rw-rw-r-- 1 ilya lab 590M Feb  2 16:18 R2_pe.fastq.gz
-rw-rw-r-- 1 ilya lab  16M Feb  2 16:18 R2_se.fastq.gz


## Align surviving paired end reads to the locus reference

In [8]:
for sample in $(ls $base_dir)
do
    sample_dir="$base_dir/$sample"
    res_dir="../results/$sample"
    read1="$sample_dir/trim/read1.fifo"
    read2="$sample_dir/trim/read2.fifo"
    mkfifo $read1
    mkfifo $read2
    if [ ! -d $res_dir ];
    then
        mkdir -p $res_dir
    fi
    zcat "$sample_dir/trim/R1_pe.fastq.gz" > $read1 &
    zcat "$sample_dir/trim/R2_pe.fastq.gz" > $read2 &

    bowtie2 -p $NUMPROC --very-sensitive -x $index \
        -1 $read1 -2 $read2 \
        | samtools view -bhS - > "$res_dir/lncRNA_locus.bam"
    rm $sample_dir/trim/*.fifo
done    


[1] 32590
[2] 32591
30534716 reads; of these:
  30534716 (100.00%) were paired; of these:
    6305496 (20.65%) aligned concordantly 0 times
    22929448 (75.09%) aligned concordantly exactly 1 time
    1299772 (4.26%) aligned concordantly >1 times
    ----
    6305496 pairs aligned concordantly 0 times; of these:
      179963 (2.85%) aligned discordantly 1 time
    ----
    6125533 pairs aligned 0 times concordantly or discordantly; of these:
      12251066 mates make up the pairs; of these:
        7385867 (60.29%) aligned 0 times
        4535375 (37.02%) aligned exactly 1 time
        329824 (2.69%) aligned >1 times
87.91% overall alignment rate
[1]-  Done                    zcat "$sample_dir/trim/R1_pe.fastq.gz" > $read1
[2]+  Done                    zcat "$sample_dir/trim/R2_pe.fastq.gz" > $read2
[1] 18985
[2] 18986
30399378 reads; of these:
  30399378 (100.00%) were paired; of these:
    6164906 (20.28%) aligned concordantly 0 times
    22900888 (75.33%) aligned concordantly exact

## Sort, index, and strand-split `.bam` files for downstream processing

In [9]:
# This function takes a sorted and indexed .bam file
# and splits it into two files corresponding to the
# alignments to + and - strand

# Example:
#
# splitstrands 201 ../../results ""

splitstrands() {
    sample=${1-"sample"}
    base_dir=${2-"../results"}
    prefix=${3-"transcripts"}
    
    result_dir="${base_dir}/${sample}"
    echo "Processing $sample... forward strand"
    # Forward strand.
    #
    # 1. alignments of the second in pair if they map to the forward strand
    # 2. alignments of the first in pair if they map to the reverse  strand
    #

    samtools view -b -f 128 -F 16 -@ $NUMPROC \
        "${result_dir}/${prefix}.sorted.bam" \
        > "${result_dir}/${prefix}.sorted.fwd1.bam"
    samtools view -b -f 80 -@ $NUMPROC \
        "${result_dir}/${prefix}.sorted.bam" \
        > "${result_dir}/${prefix}.sorted.fwd2.bam"
        
    samtools index -@ $NUMPROC "$result_dir/${prefix}.sorted.fwd1.bam"
    samtools index -@ $NUMPROC "$result_dir/${prefix}.sorted.fwd2.bam"
    
    samtools merge -@ $NUMPROC -f "$result_dir/${prefix}.sorted.fwd.bam" \
        "$result_dir/${prefix}.sorted.fwd1.bam" \
        "$result_dir/${prefix}.sorted.fwd2.bam"
    samtools index -@ $NUMPROC "$result_dir/${prefix}.sorted.fwd.bam"
    rm $result_dir/${prefix}.sorted.fwd{1,2}.bam
    rm $result_dir/${prefix}.sorted.fwd{1,2}.bam.bai
    
    # Reverse strand
    #
    # 1. alignments of the second in pair if they map to the reverse strand
    # 2. alignments of the first in pair if they map to the forward strand
    #
    echo "Processing $sample... reverse strand"
    samtools view -b -f 144 -@ $NUMPROC \
        "${result_dir}/${prefix}.sorted.bam" \
        > "${result_dir}/${prefix}.sorted.rev1.bam"
    samtools view -b -f 64 -F16 -@ $NUMPROC \
        "${result_dir}/${prefix}.sorted.bam" \
        > "${result_dir}/${prefix}.sorted.rev2.bam"
        
    samtools index -@ $NUMPROC "$result_dir/${prefix}.sorted.rev1.bam"
    samtools index -@ $NUMPROC "$result_dir/${prefix}.sorted.rev2.bam"
    
    samtools merge -@ $NUMPROC -f "$result_dir/${prefix}.sorted.rev.bam" \
        "$result_dir/${prefix}.sorted.rev1.bam" \
        "$result_dir/${prefix}.sorted.rev2.bam"
    samtools index -@ $NUMPROC "$result_dir/${prefix}.sorted.rev.bam"
    rm $result_dir/${prefix}.sorted.rev{1,2}.bam
    rm $result_dir/${prefix}.sorted.rev{1,2}.bam.bai

}

In [12]:
for sample in $(ls $result_dir)
do
    sample_dir="$result_dir/$sample"
    samtools sort -@ $NUMPROC \
        "$sample_dir/lncRNA_locus.bam" \
        -o "$sample_dir/lncRNA_locus.sorted.bam"
    samtools index -@ $NUMPROC "$sample_dir/lncRNA_locus.sorted.bam"
    splitstrands $sample "../results" "lncRNA_locus"&
done

[1] 6643
Processing yoon01... forward strand
Processing yoon01... reverse strand
[1]+  Done                    splitstrands $sample "../results" "lncRNA_locus"
[1] 13053
Processing yoon02... forward strand
Processing yoon02... reverse strand
[1]+  Done                    splitstrands $sample "../results" "lncRNA_locus"
[1] 20694


: 1

In [15]:
ls -lah ../results/yoon03

total 5.9G
drwxrwsr-x 2 ilya lab 4.0K Feb  2 17:46 .
drwxrwsr-x 5 ilya lab 4.0K Feb  2 17:24 ..
-rw-rw-r-- 1 ilya lab 2.5G Feb  2 17:22 lncRNA_locus.bam
-rw-rw-r-- 1 ilya lab 1.8G Feb  2 17:42 lncRNA_locus.sorted.bam
-rw-rw-r-- 1 ilya lab   96 Feb  2 17:43 lncRNA_locus.sorted.bam.bai
-rw-rw-r-- 1 ilya lab 880M Feb  2 17:44 lncRNA_locus.sorted.fwd.bam
-rw-rw-r-- 1 ilya lab   96 Feb  2 17:44 lncRNA_locus.sorted.fwd.bam.bai
-rw-rw-r-- 1 ilya lab 879M Feb  2 17:46 lncRNA_locus.sorted.rev.bam
-rw-rw-r-- 1 ilya lab   96 Feb  2 17:46 lncRNA_locus.sorted.rev.bam.bai
[1]+  Done                    splitstrands $sample "../results" "lncRNA_locus"


## Prepare locus coverage files

In [18]:
for sample in $(ls $result_dir)
do
    output_dir="$result_dir/$sample"
    
    (bedtools genomecov -ibam ${output_dir}/lncRNA_locus.sorted.fwd.bam \
        -d -pc | gzip -9 > ${output_dir}/lncRNA_locus.fwd.cov.gz &)
    (bedtools genomecov -ibam ${output_dir}/lncRNA_locus.sorted.rev.bam \
        -d -pc | gzip -9 > ${output_dir}/lncRNA_locus.rev.cov.gz &)
    (bedtools genomecov -ibam ${output_dir}/lncRNA_locus.sorted.bam \
        -d -pc | gzip -9 > ${output_dir}/lncRNA_locus.cov.gz &)
done